<a href="https://colab.research.google.com/github/marwankefah/Validate_SORT/blob/main/Validate_Sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MOT Data For hydration

In [1]:
!git clone https://github.com/JonathonLuiten/TrackEval

Cloning into 'TrackEval'...
remote: Enumerating objects: 821, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 821 (delta 278), reused 250 (delta 248), pack-reused 508
Receiving objects: 100% (821/821), 287.48 KiB | 11.98 MiB/s, done.
Resolving deltas: 100% (549/549), done.


In [2]:
!wget https://omnomnom.vision.rwth-aachen.de/data/TrackEval/data.zip

--2021-06-21 00:19:11--  https://omnomnom.vision.rwth-aachen.de/data/TrackEval/data.zip
Resolving omnomnom.vision.rwth-aachen.de (omnomnom.vision.rwth-aachen.de)... 137.226.34.132
Connecting to omnomnom.vision.rwth-aachen.de (omnomnom.vision.rwth-aachen.de)|137.226.34.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149957160 (143M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 143.01M  20.6MB/s    in 7.9s    

2021-06-21 00:19:20 (18.1 MB/s) - ‘data.zip’ saved [149957160/149957160]



In [ ]:
!unzip data.zip

In [4]:
!mv /content/data /content/TrackEval

In [5]:
!wget https://motchallenge.net/data/MOT15Labels.zip

--2021-06-21 00:19:27--  https://motchallenge.net/data/MOT15Labels.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1534073 (1.5M) [application/zip]
Saving to: ‘MOT15Labels.zip’

MOT15Labels.zip     100%[===================>]   1.46M  1.61MB/s    in 0.9s    

2021-06-21 00:19:29 (1.61 MB/s) - ‘MOT15Labels.zip’ saved [1534073/1534073]



In [ ]:
!unzip MOT15Labels.zip

In [7]:
# !wget https://motchallenge.net/data/MOT20Labels.zip

In [8]:
# !unzip MOT20Labels.zip

In [9]:
!git clone https://github.com/marwankefah/Kalman_Tracking_Single_Camera

Cloning into 'Kalman_Tracking_Single_Camera'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 60 (delta 18), reused 31 (delta 8), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [10]:
!git clone https://github.com/abewley/sort

Cloning into 'sort'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 202 (delta 0), reused 0 (delta 0), pack-reused 198
Receiving objects: 100% (202/202), 1.21 MiB | 16.92 MiB/s, done.
Resolving deltas: 100% (69/69), done.


## Validating SORT REPO

In [11]:
!pip install filterpy

     |████████████████████████████████| 184kB 5.1MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp37-none-any.whl size=110476 sha256=4fe873b25b1d13aef1bcdd2fcb14f1f63d7eabd8658d5a297cafecf356cea434
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built filterpy


In [14]:
!pip install tk

In [15]:
from sort.sort import Sort
import os
import glob
import time
import numpy as np
total_time = 0.0
total_frames = 0
max_age=100
min_hits=1
iou_threshold=0.4
phase = '/content/MOT15Labels/train'
seq_path='data'
output="/content/TrackEval/data/trackers/mot_challenge/MOT15-train/MPNTrack/data/"

if not os.path.exists(output):
  os.makedirs(output)
pattern = os.path.join(seq_path, phase, '*', 'det', 'det.txt')
for seq_dets_fn in glob.glob(pattern):
  mot_tracker = Sort(max_age, min_hits, iou_threshold)
  seq_dets = np.loadtxt(seq_dets_fn, delimiter=',')
  seq = seq_dets_fn[pattern.find('*'):].split(os.path.sep)[0]
  
  with open(os.path.join(output, '%s.txt'%(seq)),'w') as out_file:
    print("Processing %s."%(seq))
    for frame in range(int(seq_dets[:,0].max())):
      frame += 1 #detection and frame numbers begin at 1
      dets = seq_dets[seq_dets[:, 0]==frame, 2:7]
      dets[:, 2:4] += dets[:, 0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
      total_frames += 1

      start_time = time.time()
      trackers = mot_tracker.update(dets)
      cycle_time = time.time() - start_time
      total_time += cycle_time

      for d in trackers:
        print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file)

print("Total Tracking took: %.3f seconds for %d frames or %.1f FPS" % (total_time, total_frames, total_frames / total_time))


Processing KITTI-13.
Processing ADL-Rundle-6.
Processing ETH-Pedcross2.
Processing ETH-Bahnhof.
Processing TUD-Campus.
Processing ADL-Rundle-8.
Processing TUD-Stadtmitte.
Processing Venice-2.
Processing ETH-Sunnyday.
Processing KITTI-17.
Processing PETS09-S2L1.
Total Tracking took: 50.471 seconds for 5500 frames or 109.0 FPS


In [16]:
!python TrackEval/scripts/run_mot_challenge.py --BENCHMARK MOT15 --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL MPNTrack --METRICS HOTA CLEAR Identity VACE --USE_PARALLEL True


Eval Config:
USE_PARALLEL         : True                          
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /content/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /content/TrackEval/data/gt/mot_challenge/
TRACKERS_FOLDER      : /content/TrackEval/data/trackers/mot_challenge/
OUTPUT_FOL

## Validating Kalman Repo

In [11]:
def getPersonDet(detections):
  detectionList=[]
  for bbox,prob in detections:
    if prob >=0.5:
      detectionList.append(list(bbox2points(bbox)))
  return np.array(detectionList)

In [12]:
import os
import numpy as np
import glob
import time
from Kalman_Tracking_Single_Camera.src import tracking
from Kalman_Tracking_Single_Camera.src import helpers as hp 
phase = '/content/MOT15Labels/train'
total_time = 0.0
total_frames = 0
IOUThreshold=0.4
removeTrackAfternFramesThres=40
uncertaintyCount=1
seq_path='data'
output="/content/TrackEval/data/trackers/mot_challenge/MOT15-train/MPNTrack/data/"
# output="/content/TrackEval/data/trackers/mot_challenge/MOT16-train/MPNTrack/data/"

if not os.path.exists(output):
  os.makedirs(output)
pattern = os.path.join(seq_path, phase, '*', 'det', 'det.txt')
for seq_dets_fn in glob.glob(pattern):
  mot_tracker = tracking.KalmanTracking(IOUThreshold ,removeTrackAfternFramesThres,uncertaintyCount)
  seq_dets = np.loadtxt(seq_dets_fn, delimiter=',')
  seq = seq_dets_fn[pattern.find('*'):].split(os.path.sep)[0]
  with open(os.path.join(output, '%s.txt'%(seq)),'w') as out_file:
    print("Processing %s."%(seq))
    for frame in range(int(seq_dets[:,0].max())):
      frame += 1 #detection and frame numbers begin at 1
      dets = seq_dets[seq_dets[:, 0]==frame, 2:7]
      dets[:, 2:4] += dets[:, 0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
      total_frames += 1
      start_time = time.time()

      
      if frame<=uncertaintyCount:
        trackers= mot_tracker.match(dets[:,0:4],state="U") 
      else:
        trackers= mot_tracker.match(dets[:,0:4],state="C") 

      cycle_time = time.time() - start_time
      total_time += cycle_time

      for d in trackers:
        print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file)

    # break
print("Total Tracking took: %.3f seconds for %d frames or %.1f FPS" % (total_time, total_frames, total_frames / total_time))


Processing KITTI-13.


/content/Kalman_Tracking_Single_Camera/src/kf.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.diag(np.square(standardDev)).astype(np.float32)


Processing ADL-Rundle-6.
Processing ETH-Pedcross2.
Processing ETH-Bahnhof.
Processing TUD-Campus.
Processing ADL-Rundle-8.
Processing TUD-Stadtmitte.
Processing Venice-2.
Processing ETH-Sunnyday.
Processing KITTI-17.
Processing PETS09-S2L1.
Total Tracking took: 18.325 seconds for 5500 frames or 300.1 FPS


In [13]:
!python TrackEval/scripts/run_mot_challenge.py --BENCHMARK MOT15 --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL MPNTrack --METRICS HOTA CLEAR Identity VACE --USE_PARALLEL True


Eval Config:
USE_PARALLEL         : True                          
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /content/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /content/TrackEval/data/gt/mot_challenge/
TRACKERS_FOLDER      : /content/TrackEval/data/trackers/mot_challenge/
OUTPUT_FOL